In [10]:

import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

In [ ]:
time_precision = 4   # jagame päeva xh suurusteks plokkideks


X_train = pd.read_csv("../../data/X_train.csv")
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.ToimAeg = X_train.ToimAeg.values.astype('datetime64[D]')
X_train.ToimAeg = X_train['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_train["Aasta"] = X_train['ToimAeg'].dt.year
X_train["Kuu"] = X_train['ToimAeg'].dt.month
X_train["Päev"] = X_train['ToimAeg'].dt.day
X_train["Tund"] = X_train['ToimAeg'].dt.hour
X_train.drop(columns="ToimAeg", inplace=True)
X_train = X_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_train.head()

y_train = pd.read_csv("../../data/y_train.csv")
y_train = y_train.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_train.head()

X_valid = pd.read_csv("../../data/X_valid.csv")
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.ToimAeg = X_valid.ToimAeg.values.astype('datetime64[D]')
X_valid.ToimAeg = X_valid['ToimAeg'].apply(lambda x: x.replace(hour=x.hour-x.hour%time_precision, minute=0, second=0))
X_valid["Aasta"] = X_valid['ToimAeg'].dt.year
X_valid["Kuu"] = X_valid['ToimAeg'].dt.month
X_valid["Päev"] = X_valid['ToimAeg'].dt.day
X_valid["Tund"] = X_valid['ToimAeg'].dt.hour
X_valid.drop(columns="ToimAeg", inplace=True)
X_valid = X_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
X_valid.head()

y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.head()

X_train.head()

In [ ]:
y_train.info()
y_train.drop(columns=["Koht", "Kahjusumma"], inplace=True)
y_valid.drop(columns=["Koht", "Kahjusumma"], inplace=True)

In [ ]:
from sklearn import svm

svc_models = list()
for col in y_train.columns:
    print(col)
    svc_models.append(svm.SVC(max_iter=100, random_state=69).fit(X_train, y_train[col]))

In [ ]:
from sklearn import metrics

for model, col in zip(svc_models, y_train.columns.sort_values()):
    print(metrics.roc_auc_score(y_valid[col], model.predict(X_valid)), col)

print(len(svc_models))

In [ ]:
fitted_jalg = svm.LinearSVC(dual=True, random_state=69).fit(X_train, y_train.LIIK_JALGRATTA_MOPEEDI_VARGUS)

In [ ]:
sum(fitted_jalg.predict(X_valid))

In [ ]:
#pipeline for scv aka fancy stuff
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), svm.SVC(gamma='auto'))
clf.fit(X_train, y_train.LIIK_VARGUS)

In [11]:
megapredict = clf.predict(X_valid)

In [13]:
from sklearn import metrics
metrics.mean_squared_error(y_valid.LIIK_VARGUS, megapredict)


0.36124983992828785

SVC is quite bad and super slow
